# **Um guia prático para solução de LMIs usando Python**




---


Pedro Henrique Silva Coutinho

Universidade do Estado do Rio de Janeiro

Controle Robusto via Desigualdades Matriciais Lineares

---


Este documento apresenta alguns exemplos de problemas de análise e síntese via desigualdades matriciais lineares usando Python. A ideia é que, a partir dos exemplos aqui apresentados, o leitor seja capaz de programar qualquer outro problema baseado em LMI de interesse com conhecimentos básicos de Python.

Além da vantagem já conhecida do Python ser de cógido aberto, ferramentas como o Google Colab e Jupyther, por exemplo, possibilitam programar em Python online, dispensando instalações custosas ou acesso a licenças específicas. Além disso, ferramentas como o Google Colab permitem o compartilhamento de projetos.

Aqui, para resolver problemas de otimização baseados em LMIs, será utilizado o CVXPY, que é um pacote dedicado à solução de problemas de programação semidefinida em Python. Veja a página do projeto abaixo:
https://www.cvxpy.org/tutorial/index.html


Um material com diversos exemplos de aplicação do CVXPY é fornecido pelos autores, S. Boyd, S. Diamond, J. Park, A. Agrawal, & J. Zhang, na página abaixo:
https://stanford.edu/~boyd/papers/cvx_short_course.html


Em particular, há também um exemplo relacionado ao tópico de Controle: https://nbviewer.org/github/cvxgrp/cvx_short_course/blob/master/intro/control.ipynb

Outros exemplos de projeto de observadores usando CVXPY estão disponíveis em: https://jckantor.github.io/cbe30338-2021/04.07-Observer-Synthesis-LMI.html


O solver padrão é o SCS v3.2.2 - Splitting Conic Solver, como pode ser visto no status de retorno da solução dos problemas de otimização apresentados abaixo (basta fazer verbose=False). No entanto, é possível utilizar outros solvers, como o Mosek, mas esta configuração não será discutida aqui.

In [ ]:
import cvxpy as cp # importa a biblioteca do CVXPY
import numpy as np # importa a biblioteca NUMPY
from scipy import signal
import matplotlib.pyplot as plt
# NumPy (https://numpy.org/) é uma biblioteca para a linguagem de programação Python,
# que suporta o processamento matrizes, juntamente com uma grande coleção de
# funções matemáticas de alto nível para operar sobre estas matrizes.

## **Estabilidade de sistemas LTI em tempo contínuo**

Considere o sistema descrito por

\begin{align}
\dot{x}(t) = A x(t)
\end{align}

O objetivo aqui é checar a estabilidade do sistema acima.

**Problema:** Seja $P$ uma matriz simétrica, o sistema LTI é estável se e somente se
\begin{align*}
P > 0 \\
A^\top P + P A < 0
\end{align*}

Como exemplo, vamos considerar o sistema com a matriz $A$ dada por
\begin{align}
A =
\begin{bmatrix}
0 & 1 \\
-2 & -3
\end{bmatrix}
\end{align}

In [ ]:
## Definindo a matriz A
A = np.array([[0, 1], [-2, -3]])
print(np.linalg.eig(A)[0])

[-1. -2.]


In [ ]:
n = A.shape[0] # ordem do sistema

## Definindo a matriz P como uma matriz de variáveis de decisão
P = cp.Variable((n,n), PSD=True) # Aqui, PSD eh uma variável booleana que indica
# se a variável deverá ser simétrica e positiva definida (True), ou não (False)
# Outras estruturas de variaveis estao disponiveis em
# https://www.cvxpy.org/tutorial/advanced/index.html

## Definindo as restricoes do problema
constraints = [P >> 1e-3*np.eye(n)] # P > 0
constraints += [A.T@P + P@A << -1e-3*np.eye(n)] # A'P + PA < 0

## Definindo o custo a ser minimizado
cost = 0
objective = cp.Minimize(cost)
# Resolvendo o problema de otimizacao
prob = cp.Problem(objective,constraints)
# Uma descricao detalhada dos problemas pode ser encontrada em
# https://www.cvxpy.org/api_reference/cvxpy.problems.html

# Visualizando o status da solucao
prob.solve(verbose=False) # O status detalhado da solucao pode ser obtido
# definindo verbose = True (similar ao Yalmip no Matlab)
print('The solution is ' + prob.status) # Apresenta se a solucao
# eh factivel (optimal) ou nao (infeasible)

The solution is optimal


In [ ]:
# Para visualizar os valores das variaveis obtidas do problema anterior, basta fazer
print(P.value)

[[0.0028803  0.0007562 ]
 [0.0007562  0.00158232]]


In [ ]:
# Note que P eh de fato simetrica e positiva definida. Para visualizar os autovalores de P, basta fazer
print(np.linalg.eig(P.value)[0])

[0.00322782 0.0012348 ]


## **Instabilidade de sistemas LTI em tempo contínuo**

Considere agora o sistema com a matriz $A$ dada por
\begin{align}
A =
\begin{bmatrix}
0 & 1 \\
-1 & 2
\end{bmatrix}
\end{align}

Sabe-se que este sistema é instavel, já que possui 2 autovalores repetidos em $ 1$.

In [ ]:
## Para encontrar os autovalores da matriz A, pode-se fazer
A = np.array([[0, 1], [-1, 2]])
print(np.linalg.eig(A)[0])

[1. 1.]


In [ ]:
## Definindo a matriz A
A = np.array([[0, 1], [-1, 2]])
n = A.shape[0] # ordem do sistema
## Definindo a matriz P como uma matriz de variáveis de decisao
P = cp.Variable((n,n), PSD=True) # Aqui, PSD eh uma variable booleana que indica
# se a variavel devera ser simetrica e positiva definida (True) ou nao (False)
# Outras estruturas de variaveis estao disponiveis em
# https://www.cvxpy.org/tutorial/advanced/index.html

## Definindo as restricoes do problema
constraints = [P >> 1e-3*np.eye(n)] # P > 0
constraints += [A.T@P + P@A << -1e-3*np.eye(n)] # A'P + PA < 0

## Definindo o custo a ser minimizado
cost = 0
objective = cp.Minimize(cost)
# Resolvendo o problema de otimizacao
prob = cp.Problem(objective,constraints)
# Uma descricao detalhada dos problemas pode ser encontrada em
# https://www.cvxpy.org/api_reference/cvxpy.problems.html

# Visualizando o status da solucao
prob.solve(verbose=False) # O status detalhado da solucao pode ser obtido
# definindo verbose = True (similar ao Yalmip no Matlab)
print('The solution is ' + prob.status) # Apresenta se a solucao
# eh factivel (optimal) ou nao (infeasible)

The solution is infeasible


## **Estabilidade de sistemas LTI em tempo discreto**

Considere agora o sistema em tempo discreto
\begin{align}
x(k+1) = A x(k)
\end{align}
com a matriz $A$ dada por
\begin{align}
A =
\begin{bmatrix}
0 & 1 \\
0 & 0
\end{bmatrix}
\end{align}

Sabe-se que este sistema é estável, já que possui 2 autovalores repetidos em $ 0$ (contidos no círculo de raio unitário).

In [ ]:
A = np.array([[0, 1], [0, -2]])
print(np.linalg.eig(A)[0])

[ 0. -2.]


In [ ]:
## Definindo a matriz A
A = np.array([[0, 1], [0, -2]])
n = A.shape[0] # ordem do sistema

## Definindo a matriz P como uma matriz de variáveis de decisão
P = cp.Variable((n,n), PSD=True) # Aqui, PSD eh uma variável booleana que indica
# se a variável deverá ser simétrica e positiva definida (True), ou não (False)
# Outras estruturas de variaveis estao disponiveis em
# https://www.cvxpy.org/tutorial/advanced/index.html

## Definindo as restricoes do problema
constraints = [P >> 1e-3*np.eye(n)] # P > 0
constraints += [A.T@P@A - P << -1e-3*np.eye(n)] # A'PA - P < 0

## Definindo o custo a ser minimizado
cost = 0
objective = cp.Minimize(cost)
# Resolvendo o problema de otimizacao
prob = cp.Problem(objective,constraints)
# Uma descricao detalhada dos problemas pode ser encontrada em
# https://www.cvxpy.org/api_reference/cvxpy.problems.html

# Visualizando o status da solucao
prob.solve(verbose=False) # O status detalhado da solucao pode ser obtido
# definindo verbose = True (similar ao Yalmip no Matlab)
print('The solution is ' + prob.status) # Apresenta se a solucao
# eh factivel (optimal) ou nao (infeasible)

The solution is infeasible


## **Projeto de controle por realimentação de estados para sistemas LTI em tempo contínuo**

Considere o sistema descrito por

\begin{align}
\dot{x}(t) = A x(t) + B u(t)
\end{align}

O objetivo aqui é projetar o ganho $K$ da lei de controle
\begin{align}
u(t) = K x(t)
\end{align}
tal que o sistema em malha fechada seja estável.

**Problema:** Seja $Q$ uma matriz simétrica e $L$ uma matriz cheia. Se o problema de factibilidade sujeito às LMIs abaixo
\begin{align*}
Q > 0 \\
Q A^\top +  A Q + B L + L^\top B^\top < 0
\end{align*}
for satisfeito, então o sistema em malha fechada é estável.

Como exemplo, vamos considerar o sistema com as matrizes
\begin{align}
A =
\begin{bmatrix}
0 & 1 \\
-1 & 2
\end{bmatrix}, \quad
B =
\begin{bmatrix}
0 \\ 1
\end{bmatrix}
\end{align}

In [ ]:
## Definindo as matrizes do sistems
A = np.array([[0, 1], [-1, 2]])
B = np.array([[0], [1]])
print(np.linalg.eig(A)[0])

[1. 1.]


In [ ]:
n = A.shape[0] # ordem do sistema
m = B.shape[1] # numero de entradas
## Definindo a matriz P como uma matriz de variáveis de decisao
Q = cp.Variable((n,n), PSD=True) # Aqui, PSD eh uma variable booleana que indica
# se a variavel devera ser simetrica e positiva definida (True) ou nao (False)
# Outras estruturas de variaveis estao disponiveis em
# https://www.cvxpy.org/tutorial/advanced/index.html
L = cp.Variable((m,n), PSD=False)

## Definindo as restricoes do problema
constraints = [Q >> 1e-3*np.eye(n)] # Q > 0
constraints += [Q@A.T + A@Q + B@L + L.T@B.T <<  -1e-3*np.eye(n)] # Q A' + A Q + BL + L'B' < 0

## Definindo o custo a ser minimizado
cost = 0
objective = cp.Minimize(cost)
# Resolvendo o problema de otimizacao
prob = cp.Problem(objective,constraints)
# Uma descricao detalhada dos problemas pode ser encontrada em
# https://www.cvxpy.org/api_reference/cvxpy.problems.html

# Visualizando o status da solucao
prob.solve(verbose=False) # O status detalhado da solucao pode ser obtido
# definindo verbose = True (similar ao Yalmip no Matlab)
print('The solution is ' + prob.status) # Apresenta se a solucao
# eh factivel (optimal) ou nao (infeasible)

K = L.value@np.linalg.inv(Q.value)
print(K)
print(np.linalg.eig(A+B@K)[0])

The solution is optimal
[[-0.31111802 -2.77465503]]
[-0.38732751+1.07754138j -0.38732751-1.07754138j]


## **Projeto de controle por realimentação de estados para sistemas LTI em tempo discreto**

Considere o sistema descrito por

\begin{align}
{x}(k+1) = A x(k) + B u(k)
\end{align}

O objetivo aqui é projetar o ganho $K$ da lei de controle
\begin{align}
u(k) = K x(k)
\end{align}
tal que o sistema em malha fechada seja estável.

**Problema:** Seja $X$ uma matriz simétrica e $L$ uma matriz cheia. Se o problema de factibilidade sujeito às LMIs abaixo
\begin{align*}
X > 0 \\
\begin{bmatrix}
-X & XA^\top + L^\top B^\top \\
AX+BL & -X
\end{bmatrix} < 0
\end{align*}
for satisfeito, então o sistema em malha fechada é estável.

Como exemplo, vamos considerar o sistema com as matrizes
\begin{align}
A =
\begin{bmatrix}
0 & 1 \\
1 & 2
\end{bmatrix}, \quad
B =
\begin{bmatrix}
0 \\ 1
\end{bmatrix}
\end{align}

In [ ]:
## Definindo as matrizes do sistems
A = np.array([[0, 1], [1, 2]])
B = np.array([[0], [1]])

In [ ]:
print(np.linalg.eig(A)[0])

[-0.41421356  2.41421356]


In [ ]:
n = A.shape[0] # ordem do sistema
m = B.shape[1] # numero de entradas
## Definindo a matriz P como uma matriz de variáveis de decisao
X = cp.Variable((n,n), PSD=True) # Aqui, PSD eh uma variable booleana que indica
# se a variavel devera ser simetrica e positiva definida (True) ou nao (False)
# Outras estruturas de variaveis estao disponiveis em
# https://www.cvxpy.org/tutorial/advanced/index.html
L = cp.Variable((m,n), PSD=False)

## Definindo as restricoes do problema
constraints = [X >> 1e-3*np.eye(n)] # P > 0
constraints += [cp.bmat([[-X, X@A.T + L.T@B.T],
                         [A@X + B@L, -X]]) <<  -1e-3*np.eye(2*n)] # A'P + PA + BL + L'B' < 0

## Definindo o custo a ser minimizado
cost = 0
objective = cp.Minimize(cost)
# Resolvendo o problema de otimizacao
prob = cp.Problem(objective,constraints)
# Uma descricao detalhada dos problemas pode ser encontrada em
# https://www.cvxpy.org/api_reference/cvxpy.problems.html

# Visualizando o status da solucao
prob.solve(verbose=False) # O status detalhado da solucao pode ser obtido
# definindo verbose = True (similar ao Yalmip no Matlab)
print('The solution is ' + prob.status) # Apresenta se a solucao
# eh factivel (optimal) ou nao (infeasible)

K = L.value@np.linalg.inv(X.value)
print(K)
print(np.linalg.eig(A+B@K)[0])

The solution is optimal
[[-0.99999562 -2.00000084]]
[ 0.00209342 -0.00209426]


# Análise de estabilidade quadrática de sistemas politópicos em tempo contínuo

Considere o sistema descrito por

\begin{align}
\dot{x}(t) = A(\alpha(t)) x(t)
\end{align}

onde $\alpha$ é um parâmetro que pertence ao *simplex*

\begin{align}
\Lambda = \left\lbrace\alpha \in \mathbb{R}^N : \sum_{i = 1}^N \alpha_i = 1, \; \alpha_i \geq 0\right\rbrace
\end{align}

**Problema:** Seja $P$ uma matriz simétrica. Se o problema de factibilidade sujeito às LMIs abaixo
\begin{align*}
P &> 0 \\
A_i^\top P +  P A_i &< 0, \quad \forall i = 1,2,\ldots, N,
\end{align*}
for satisfeito, então o sistema é quadraticamente estável.

Como exemplo, vamos considerar o sistema com as matrizes
\begin{align}
A_1 =
\begin{bmatrix}
0 & 1 \\
-2 & -1
\end{bmatrix}, \quad
A_2 =
\begin{bmatrix}
-1 & 0  \\
3 & -1
\end{bmatrix}
\end{align}

In [ ]:
## Definindo as matrizes Ai
A = np.array([[[0, 1], [-2, -1]],[[0,1], [-3, -1]]])
n = A.shape[1] # ordem do sistema
N = A.shape[0] # numero de vertices

## Definindo a matriz P como uma matriz de variáveis de decisao
P = cp.Variable((n,n), PSD=True) # Aqui, PSD eh uma variable booleana que indica
# se a variavel devera ser simetrica e positiva definida (True) ou nao (False)
# Outras estruturas de variaveis estao disponiveis em
# https://www.cvxpy.org/tutorial/advanced/index.html

## Definindo as restricoes do problema
constraints = [P >> 1e-3*np.eye(n)] # P > 0
for i in range(N):
  constraints += [A[i].T@P + P@A[i] << -1e-3*np.eye(n)] # A'P + PA < 0

## Definindo o custo a ser minimizado
cost = 0
objective = cp.Minimize(cost)
# Resolvendo o problema de otimizacao
prob = cp.Problem(objective,constraints)
# Uma descricao detalhada dos problemas pode ser encontrada em
# https://www.cvxpy.org/api_reference/cvxpy.problems.html

# Visualizando o status da solucao
prob.solve(verbose=False) # O status detalhado da solucao pode ser obtido
# definindo verbose = True (similar ao Yalmip no Matlab)
print('The solution is ' + prob.status) # Apresenta se a solucao
# eh factivel (optimal) ou nao (infeasible)

The solution is optimal


In [ ]:
print(A.shape[1])

2


# Projeto de controlador com ganho escalonado para estabilização de sistemas politópicos em tempo contínuo

Considere o sistema descrito por

\begin{align}
\dot{x}(t) = A(\alpha(t)) x(t) + B(\alpha(t))u(t)
\end{align}

e a seguinte lei de controle com ganho escalonado
\begin{align}
 u(t) = K(\alpha(t)) x(t)
\end{align}

**Problema:** Seja $Q$ uma matriz simétrica e $L_i$, $i = 1,\ldots,N$, matrizes cheias. Se o problema de factibilidade sujeito às LMIs abaixo for satisfeito
\begin{align*}
Q &> 0 \\
\Gamma_{ii} &< 0, \quad i = j, \\
\Gamma_{ij} + \Gamma_{ji} &< 0, \quad i < j, \quad \forall i, j = 1,2,\ldots, N,
\end{align*}
onde $\Gamma_{ij}=Q A_i^\top +  A_iQ + B_i L_j + L_j^\top B_i^\top$,
então o sistema em malha fechada é assintoticamente estável.

Como exemplo, vamos considerar o sistema com as matrizes
\begin{align}
A_1 =
\begin{bmatrix}
0 & 1 \\
-2 & -1
\end{bmatrix}, \quad
A_2 =
\begin{bmatrix}
-1 & 0  \\
3 & -1
\end{bmatrix}, \quad
B_{1} = B_{2} =
\begin{bmatrix}
0 \\ 1
\end{bmatrix}
\end{align}

In [ ]:
## Definindo as matrizes A_i
A = np.array([[[0, 1], [-2, -1]], [[-1, 0], [3, -1]]])
B = np.array([[[0], [1]], [[0], [1]]])
n = A.shape[0] # ordem do sistema
m = B.shape[2] # numero de entradas
N = A.shape[0] # numero de vertices

## Definindo a matriz P como uma matriz de variáveis de decisao
Q = cp.Variable((n,n), PSD=True) # Aqui, PSD eh uma variable booleana que indica
# se a variavel devera ser simetrica e positiva definida (True) ou nao (False)
# Outras estruturas de variaveis estao disponiveis em
# https://www.cvxpy.org/tutorial/advanced/index.html
L = {}
for i in range(N):
  L[i] = cp.Variable((m,n), PSD=False)

## Definindo Gamma
Gamma = {} # inicializa Gamma
for i in range(N):
  for j in range(N):
    Gamma[i,j] = Q@A[i].T + A[i]@Q + B[i]@L[j] + L[j].T@B[i].T

## Definindo as restricoes do problema
constraints = [Q >> 1e-3*np.eye(n)] # P > 0
for i in range(N):
  for j in range(N):
    if i == j:
      constraints += [Gamma[i,i] << -1e-3*np.eye(n)] # Gamma_{ii} < 0
    else:
      constraints += [Gamma[i,j] + Gamma[j,i] << -1e-3*np.eye(n)] # Gamma_{ij} + Gamma_{ji} < 0

## Definindo o custo a ser minimizado
cost = 0
objective = cp.Minimize(cost)
# Resolvendo o problema de otimizacao
prob = cp.Problem(objective,constraints)
# Uma descricao detalhada dos problemas pode ser encontrada em
# https://www.cvxpy.org/api_reference/cvxpy.problems.html

# Visualizando o status da solucao
prob.solve(verbose=False) # O status detalhado da solucao pode ser obtido
# definindo verbose = True (similar ao Yalmip no Matlab)
print('The solution is ' + prob.status) # Apresenta se a solucao
# eh factivel (optimal) ou nao (infeasible)

K = {}
for i in range(N):
  K[i] = L[i].value@np.linalg.inv(Q.value)

print(K)

The solution is optimal
{0: array([[ 0.39300932, -0.01843558]]), 1: array([[-3.85105296,  0.26549701]])}


# O que mais pode ser feito?

Este documento apresentou apenas os primeiros passos para resolver LMIs em Python. Apesar disso, foi possível resolver problemas de análise e síntese que estamos habituados a implementar no Matlab.

Há duas extensões que devem ser investigadas posteriormente:
 - instalação do Mosek e configuração para utilizá-lo com o CVXPY;
 - desenvolvimento de funções para aplicar relaxações polinomiais e Lema de Pólya para fornecer condições progressivamente menos conservadoras.